<a href="https://colab.research.google.com/github/satuelisa/NLPF/blob/main/NLPF_09_P.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import nltk
from nltk.chat.util import Chat, reflections

What are *reflections*?

In [65]:
print(reflections)

{'i am': 'you are', 'i was': 'you were', 'i': 'you', "i'm": 'you are', "i'd": 'you would', "i've": 'you have', "i'll": 'you will', 'my': 'your', 'you are': 'I am', 'you were': 'I was', "you've": 'I have', "you'll": 'I will', 'your': 'my', 'yours': 'mine', 'you': 'me', 'me': 'you', 'hello': 'hi', 'bye': 'see you', 'o/': '\\o', '>': '<'}


We can add stuff.

In [69]:
reflections['myself'] = 'yourself'

We need to structure what will the bot respond to diverse prompts.

In [74]:
intro = ['Pleased to meet you, %1.', "Hello, %1, how's it going?",]
smalltalk = [ 'I am well, thanks', 'Okay',]
identify = ['I am a bot',]
positive = [ 'Nice', 'Excellent', 'Way to go', 'Good for you',]
negative = [ 'That sucks', 'Sorry to hear that', ':(',]
default = [ 'Wow', 'I hear you',]
questionResponse = [ # will be matched in this order
    [ r'my name is (.*)', intro ],
    [ r'I am (.*)', intro ],
    [ r'this is (.*)', intro ],
    [ r'who am I talking to|who is this', identify ],
    [ r"how are you|how about you|what's up", smalltalk ],
    [ r'not too bad|fine|well|great', positive ],
    [ r'bad|sucks', negative ],
    [ r'bonjour|hi|whazzup (.*)', ['ohai', 'Bonjour, ça va?',] ], 
    [ r'sorry (.*)', ['no biggie',] ],
    [ r'close|farewell (.*)', ['It was a pleasure', 'See you later',] ],
    [ r'(.*)\!', ['%1, for sure'] ],
    [ r'(.*)\?', ['whoah, %1!',] ],
    [ r'(.*)', default ]
]

Let's start a chat :)

In [75]:
print('At your service, mate.')
chat = Chat(questionResponse, reflections)
chat.converse(quit = 'close') # typing close will make it stop expecting more messages

At your service, mate.
>the weather sucks today
Wow
>who is this?
I am a bot
>I am elisa
Hello, elisa, how's it going?
>well, thanks
Way to go
>nice to meet you
I hear you
>my cat climbed on my table
Wow
>do you have a cat?
whoah, do me have a cat!
>close
It was a pleasure


To be continued...